Task 1. MongoDB Data Model
Design a suitable data model to support the efficient querying of the two data sets in MongoDB. Justify the data model design.
- Example of data model
- Justification for choosing that data model

The data model that would be suitable for **climate_historic.csv** is by using **referencing** method. 

An example of the data model would be as such:<br>
The data model would be consisted of 2 collections: <br>
a. Station collection <br>

{
  >station: 948700, 
  <br>
  dates: ["31-12-2017", "2-1-2018", ..., "19-2-2018"]

} 
<br>
b. Date collection<br>

{
  >date: "31-12-2017",
  <br>
  air_temperature_celcius: 19,
  <br>
  relative_humidity: 56.8,
  <br>
  windspeed_knots: 7.9,
  <br>
  max_wind_speed: 11.1,
  <br>
  precipitation: "0.001",
  <br>
  GHI_w/m2: 154
  <br>

}<br>
{
  >date: "2-1-2018",
  <br>
  air_temperature_celcius: 15,
  <br>
  relative_humidity: 50.7,
  <br>
  windspeed_knots: 9.2,
  <br>
  max_wind_speed: 13,
  <br>
  precipitation: "0.02G",
  <br>
  GHI_w/m2: 128
  <br>

}<br>

and so on...

The justification to this data model is made through visual inspection as I have observed from the csv file that a station reports a range of dates where each date reports different values for other columns such as air_temperature_celcius, relative_humidity, etc. Furthermore, by separating the data model into 2 documents, it allows modularity for addition of more stations and it seems that each station is in charge of a particular date. Hence, by designing the data model as such, we can easily access the range of dates that a station is assigned to by referencing as well as its respective reported values in other columns by aggregation.

The data model that would be suitable for **hotspot_historic.csv** is by using **embedding** method.

An example of the data model would be as such:
<br>
{
  >latitude: -37.966,
  <br>
  longitude: 145.051,
  <br>
  reports: 
  <br>
  >[
    >>{
      <br>
      datetime: 2018-12-27T04:16:51, <br>
      confidence: 78, <br>
      date: 27-12-2018, <br>
      surface_temperature_celcius: 68 <br>
  }, <br> and so on...

  >] <br>

}

The reasoning to such a design is because at a particular "location" marked by its latitude and longitude, we may be recording the surface_temperature_celcius value for every second for a particular date and time where the temperature may maintain but the confidence values may differ. As such, I have designed it for a particular "location", we may record the confidence level and surface_temperature_celcius for every second for a particular date.

Task 2. Querying MongoDB using PyMongo
1. Write a python program that will read the data of hotspot_historic.csv and climate_historic.csv and load them to a new database. The collection of the database will be based on the document model you have designed in Task A1.

- Please use a csv library to read the files.
<br>
- Please DO NOT use the mongo aggregation query to do this task.

# Setting up MongoClient

In [1]:
import pymongo
from pymongo import MongoClient

client = MongoClient()
db = client.asgn_3182

# Reading csv files

I will be using the pandas library to read the csv files.

In [2]:
import pandas as pd

climate = pd.read_csv("climate_historic.csv")
hotspot = pd.read_csv("hotspot_historic.csv")

In [3]:
climate

station        date  air_temperature_celcius  relative_humidity  \
0     948700  31/12/2017                       19               56.8   
1     948700    2/1/2018                       15               50.7   
2     948700    3/1/2018                       16               53.6   
3     948700    4/1/2018                       24               61.6   
4     948700    5/1/2018                       24               62.3   
5     948700    6/1/2018                       26               60.8   
6     948700    7/1/2018                       32               54.1   
7     948700    8/1/2018                       20               57.7   
8     948700    9/1/2018                       19               56.3   
9     948700   10/1/2018                       20               57.0   
10    948700   11/1/2018                       18               53.3   
11    948700   12/1/2018                       22               56.3   
12    948700   13/1/2018                       19               54.1   
13    948700   14/1/2018                       17               50.4   
14    948700   15/1/2018                       16               50.8   
15    948700   16/1/2018                       24               55.4   
16    948700   17/1/2018                       24               53.4   
17    948700   18/1/2018                       16               46.7   
18    948700   19/1/2018                       19               57.2   
19    948700   20/1/2018                       16               50.9   
20    948700   21/1/2018                       17               52.5   
21    948700   22/1/2018                       23               59.0   
22    948700   23/1/2018                       25               58.3   
23    948700   24/1/2018                       16               48.2   
24    948700   25/1/2018                       17               50.8   
25    948700   26/1/2018                       18               53.5   
26    948700   27/1/2018                       19               56.1   
27    948700   28/1/2018                       18               57.0   
28    948700   29/1/2018                       22               58.0   
29    948700   30/1/2018                       20               55.8   
..       ...         ...                      ...                ...   
336   948702   3/12/2018                       14               48.2   
337   948702   4/12/2018                       14               48.2   
338   948702   5/12/2018                       17               53.0   
339   948702   6/12/2018                       16               51.7   
340   948702   7/12/2018                       15               51.0   
341   948702   8/12/2018                       15               48.1   
342   948702   9/12/2018                       17               53.6   
343   948702  10/12/2018                       17               53.5   
344   948702  11/12/2018                       17               52.0   
345   948702  12/12/2018                       19               55.3   
346   948702  13/12/2018                       28               49.9   
347   948702  14/12/2018                       17               52.5   
348   948702  15/12/2018                       18               52.0   
349   948702  16/12/2018                       18               53.7   
350   948702  17/12/2018                       21               58.7   
351   948702  18/12/2018                       25               60.9   
352   948702  19/12/2018                       21               57.4   
353   948702  20/12/2018                       17               51.2   
354   948702  21/12/2018                       17               52.5   
355   948702  22/12/2018                       19               55.8   
356   948702  23/12/2018                       19               53.9   
357   948702  24/12/2018                       16               47.0   
358   948702  25/12/2018                       17               50.4   
359   948702  26/12/2018                     

In [4]:
hotspot

latitude  longitude             datetime  confidence        date  \
0     -37.9660   145.0510  2018-12-27T04:16:51          78  27/12/2018   
1     -35.5410   143.3110  2018-12-27T00:02:15          82  27/12/2018   
2     -35.5540   143.3070  2018-12-27T00:02:15          67  27/12/2018   
3     -35.5430   143.3160  2018-12-27T00:02:14          86  27/12/2018   
4     -37.7080   145.1000  2018-12-25T04:29:08          80  25/12/2018   
5     -35.6460   142.2820  2018-12-24T13:12:01          65  24/12/2018   
6     -38.0750   143.7600  2018-12-21T04:53:29          72  21/12/2018   
7     -37.6360   149.3300  2018-12-16T15:38:39          94  16/12/2018   
8     -37.6240   149.3320  2018-12-16T15:38:39          69  16/12/2018   
9     -37.8200   142.3230  2018-12-16T04:35:15          62  16/12/2018   
10    -37.8750   142.5100  2018-12-16T04:35:13          93  16/12/2018   
11    -37.6080   149.3280  2018-12-16T04:34:58          84  16/12/2018   
12    -37.6130   149.3050  2018-12-16T04:34:58          95  16/12/2018   
13    -37.6440   149.3460  2018-12-16T04:34:57          59  16/12/2018   
14    -37.6240   149.3140  2018-12-16T04:34:57          90  16/12/2018   
15    -37.6610   149.3270  2018-12-16T04:34:57          75  16/12/2018   
16    -37.6490   149.3220  2018-12-16T04:34:57          79  16/12/2018   
17    -38.0570   144.2110  2018-12-16T00:21:12          93  16/12/2018   
18    -38.0460   144.2010  2018-12-16T00:21:11          60  16/12/2018   
19    -38.0480   144.2130  2018-12-16T00:21:11          82  16/12/2018   
20    -37.6510   149.3450  2018-12-16T00:20:53          82  16/12/2018   
21    -37.6300   149.3350  2018-12-16T00:20:53          53  16/12/2018   
22    -37.6270   149.3300  2018-12-15T13:17:17          92  15/12/2018   
23    -37.6580   149.3390  2018-12-15T13:17:17          78  15/12/2018   
24    -37.6230   149.3230  2018-12-15T13:17:17          51  15/12/2018   
25    -38.0380   142.9860  2018-12-15T01:16:23          65  15/12/2018   
26    -37.9500   142.3660  2018-12-14T04:47:25          92  14/12/2018   
27    -38.2310   147.1720  2018-12-13T04:04:19          83  13/12/2018   
28    -37.9030   145.2500  2018-12-12T00:45:38          53  12/12/2018   
29    -37.9870   144.0050  2018-12-10T00:57:59          50  10/12/2018   
...        ...        ...                  ...         ...         ...   
2638  -36.9980   148.1994  2018-03-14T04:19:30          73   14/3/2018   
2639  -36.9959   148.2118  2018-03-14T04:16:30          95   14/3/2018   
2640  -34.3654   141.5430  2018-03-14T04:16:20          89   14/3/2018   
2641  -37.0228   148.1996  2018-03-14T04:14:40          73   14/3/2018   
2642  -34.3539   141.5629  2018-03-14T04:14:40          81   14/3/2018   
2643  -34.3554   141.5413  2018-03-14T04:14:30          96   14/3/2018   
2644  -34.3555   141.5495  2018-03-14T04:14:30          94   14/3/2018   
2645  -37.0286   148.1550  2018-03-13T23:58:50          56   13/3/2018   
2646  -37.0316   148.1519  2018-03-13T12:57:00          52   13/3/2018   
2647  -37.4157   147.0084  2018-03-12T04:33:50          94   12/3/2018   
2648  -37.0193   148.1459  2018-03-12T04:29:50          84   12/3/2018   
2649  -37.4229   147.0270  2018-03-12T04:28:20         100   12/3/2018   
2650  -37.0055   148.1582  2018-03-12T04:28:00          80   12/3/2018   
2651  -37.4128   147.0242  2018-03-12T04:27:20          85   12/3/2018   
2652  -37.2230   147.9431  2018-03-10T04:48:40         100   10/3/2018   
2653  -37.2252   147.9363  2018-03-10T04:46:20         100   10/3/2018   
2654  -37.2284   147.9187  2018-03-10T04:45:30          94   10/3/2018   
2655  -37.6572   142.0703  2018-03-10T04:45:30          97   10/3/2018   
2656  -37.6592   142.0580  2018-03-10T04:43:50          81   10/3/2018   
2657  -36.2544   148.0353  2018-03-10T04:43:00          55   10/3/2018   
2658  -37.2197   147.9621  2018-03-10T04:42:30          54   10/3/2018   
2659  -37.2128   147.9308  2018-03-10T04:39:00          74   10/3/2018   
2660  -37.

# Creation of collection for climate

Pre-processing of data to create the document

In [5]:
# First I will get the unique number of stations
climate.station.unique()

array([948700, 948701, 948702])

In [6]:
# From the above output, it seems that I may need 3 arrays, as such I will create 3 variables for the arrays
arr1 = []
arr2 = []
arr3 = []
# Create a variable of the unique stations
unique_val = climate.station.unique()
# The arrays will store the dates of the document
for i in range(len(climate)):
    if int(climate.station[i]) == int(unique_val[0]):
        temp = climate.date[i].replace("/", "-")
        arr1.append(temp)
    elif int(climate.station[i]) == int(unique_val[1]):
        temp = climate.date[i].replace("/", "-")
        arr2.append(temp)
    else:
        temp = climate.date[i].replace("/", "-")
        arr3.append(temp)
# With this loop, I will be able to divide the dates to its respective station

In [7]:
# Drop the collection before creating a new collection to prevent
# double insertion
db.climate_station.drop()
climate_station = db.climate_station

In [8]:
# Since the number of stations is small here, I will just manually append it
stations = [
{
    str(climate.columns[0]):int(unique_val[0]),
    str(climate.columns[1]): arr1
}, 
{
    str(climate.columns[0]):int(unique_val[1]),
    str(climate.columns[1]): arr2
}, 
{
    str(climate.columns[0]):int(unique_val[2]),
    str(climate.columns[1]): arr3
}]
result = climate_station.insert_many(stations)

In [9]:
# Check the document of station
from pprint import pprint

cursor = climate_station.find({})
for document in cursor: 
    pprint(document)

{'_id': ObjectId('60aa1eb59343697468b4cca4'),
 'date': ['31-12-2017',
          '2-1-2018',
          '3-1-2018',
          '4-1-2018',
          '5-1-2018',
          '6-1-2018',
          '7-1-2018',
          '8-1-2018',
          '9-1-2018',
          '10-1-2018',
          '11-1-2018',
          '12-1-2018',
          '13-1-2018',
          '14-1-2018',
          '15-1-2018',
          '16-1-2018',
          '17-1-2018',
          '18-1-2018',
          '19-1-2018',
          '20-1-2018',
          '21-1-2018',
          '22-1-2018',
          '23-1-2018',
          '24-1-2018',
          '25-1-2018',
          '26-1-2018',
          '27-1-2018',
          '28-1-2018',
          '29-1-2018',
          '30-1-2018',
          '31-1-2018',
          '1-2-2018',
          '2-2-2018',
          '3-2-2018',
          '4-2-2018',
          '5-2-2018',
          '6-2-2018',
          '7-2-2018',
          '8-2-2018',
          '9-2-2018',
          '10-2-2018',
          '11-2-2018',
    

# Creation of collection for climate_date

In [10]:
# Drop the collection before creating a new collection to prevent
# double insertion
db.climate_date.drop()
climate_date = db.climate_date

In [11]:
for i in range(len(climate)):
    temp = climate[climate.columns[1]][i].replace("/", "-")
    date = {
        climate.columns[1]:str(temp),
        climate.columns[2]:int(climate[climate.columns[2]][i]),
        climate.columns[3]:float(climate[climate.columns[3]][i]),
        climate.columns[4]:float(climate[climate.columns[4]][i]),
        climate.columns[5]:float(climate[climate.columns[5]][i]),
        climate.columns[6]:str(climate[climate.columns[6]][i]),
        climate.columns[7]:int(climate[climate.columns[7]][i])
    }
    climate_date.insert_one(date)

In [12]:
# Check the document of station
from pprint import pprint

cursor = climate_date.find({})
for document in cursor: 
    pprint(document)

{'GHI_w/m2': 154,
 '_id': ObjectId('60aa1eb69343697468b4cca7'),
 'air_temperature_celcius': 19,
 'date': '31-12-2017',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.8,
 'windspeed_knots': 7.9}
{'GHI_w/m2': 128,
 '_id': ObjectId('60aa1eb69343697468b4cca8'),
 'air_temperature_celcius': 15,
 'date': '2-1-2018',
 'max_wind_speed': 13.0,
 'precipitation ': ' 0.02G',
 'relative_humidity': 50.7,
 'windspeed_knots': 9.2}
{'GHI_w/m2': 133,
 '_id': ObjectId('60aa1eb69343697468b4cca9'),
 'air_temperature_celcius': 16,
 'date': '3-1-2018',
 'max_wind_speed': 15.0,
 'precipitation ': ' 0.00G',
 'relative_humidity': 53.6,
 'windspeed_knots': 8.1}
{'GHI_w/m2': 186,
 '_id': ObjectId('60aa1eb69343697468b4ccaa'),
 'air_temperature_celcius': 24,
 'date': '4-1-2018',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 61.6,
 'windspeed_knots': 7.7}
{'GHI_w/m2': 185,
 '_id': ObjectId('60aa1eb69343697468b4ccab'),
 'air_temperature_celcius': 24,
 'date

 'air_temperature_celcius': 19,
 'date': '8-4-2018',
 'max_wind_speed': 21.0,
 'precipitation ': ' 0.47G',
 'relative_humidity': 52.8,
 'windspeed_knots': 12.5}
{'GHI_w/m2': 96,
 '_id': ObjectId('60aa1eb69343697468b4cd09'),
 'air_temperature_celcius': 11,
 'date': '9-4-2018',
 'max_wind_speed': 22.9,
 'precipitation ': ' 0.59G',
 'relative_humidity': 48.2,
 'windspeed_knots': 10.8}
{'GHI_w/m2': 120,
 '_id': ObjectId('60aa1eb69343697468b4cd0a'),
 'air_temperature_celcius': 14,
 'date': '10-4-2018',
 'max_wind_speed': 20.0,
 'precipitation ': ' 0.01G',
 'relative_humidity': 50.4,
 'windspeed_knots': 12.8}
{'GHI_w/m2': 118,
 '_id': ObjectId('60aa1eb69343697468b4cd0b'),
 'air_temperature_celcius': 14,
 'date': '11-4-2018',
 'max_wind_speed': 13.0,
 'precipitation ': ' 0.00G',
 'relative_humidity': 52.5,
 'windspeed_knots': 7.0}
{'GHI_w/m2': 118,
 '_id': ObjectId('60aa1eb69343697468b4cd0c'),
 'air_temperature_celcius': 14,
 'date': '12-4-2018',
 'max_wind_speed': 11.1,
 'precipitation ': ' 

{'GHI_w/m2': 90,
 '_id': ObjectId('60aa1eb69343697468b4cd59'),
 'air_temperature_celcius': 10,
 'date': '28-6-2018',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 43.7,
 'windspeed_knots': 7.2}
{'GHI_w/m2': 91,
 '_id': ObjectId('60aa1eb69343697468b4cd5a'),
 'air_temperature_celcius': 10,
 'date': '29-6-2018',
 'max_wind_speed': 15.0,
 'precipitation ': ' 0.13B',
 'relative_humidity': 42.1,
 'windspeed_knots': 9.4}
{'GHI_w/m2': 48,
 '_id': ObjectId('60aa1eb69343697468b4cd5b'),
 'air_temperature_celcius': 5,
 'date': '30-6-2018',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00A',
 'relative_humidity': 34.9,
 'windspeed_knots': 4.3}
{'GHI_w/m2': 49,
 '_id': ObjectId('60aa1eb69343697468b4cd5c'),
 'air_temperature_celcius': 5,
 'date': '1-7-2018',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 33.1,
 'windspeed_knots': 5.8}
{'GHI_w/m2': 60,
 '_id': ObjectId('60aa1eb69343697468b4cd5d'),
 'air_temperature_celcius': 6,
 'date': '2-7

{'GHI_w/m2': 102,
 '_id': ObjectId('60aa1eb69343697468b4cdb3'),
 'air_temperature_celcius': 11,
 'date': '26-9-2018',
 'max_wind_speed': 8.9,
 'precipitation ': ' 0.00G',
 'relative_humidity': 40.8,
 'windspeed_knots': 6.4}
{'GHI_w/m2': 134,
 '_id': ObjectId('60aa1eb69343697468b4cdb4'),
 'air_temperature_celcius': 15,
 'date': '27-9-2018',
 'max_wind_speed': 19.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 44.8,
 'windspeed_knots': 8.3}
{'GHI_w/m2': 117,
 '_id': ObjectId('60aa1eb69343697468b4cdb5'),
 'air_temperature_celcius': 13,
 'date': '28-9-2018',
 'max_wind_speed': 15.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 43.6,
 'windspeed_knots': 9.0}
{'GHI_w/m2': 128,
 '_id': ObjectId('60aa1eb69343697468b4cdb6'),
 'air_temperature_celcius': 14,
 'date': '29-9-2018',
 'max_wind_speed': 18.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 41.6,
 'windspeed_knots': 13.1}
{'GHI_w/m2': 101,
 '_id': ObjectId('60aa1eb69343697468b4cdb7'),
 'air_temperature_celcius': 11,
 'da

 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 52.5,
 'windspeed_knots': 5.7}
{'GHI_w/m2': 155,
 '_id': ObjectId('60aa1eb69343697468b4ce0a'),
 'air_temperature_celcius': 19,
 'date': '22-12-2018',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 55.8,
 'windspeed_knots': 6.0}
{'GHI_w/m2': 158,
 '_id': ObjectId('60aa1eb69343697468b4ce0b'),
 'air_temperature_celcius': 19,
 'date': '23-12-2018',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 53.9,
 'windspeed_knots': 8.3}
{'GHI_w/m2': 141,
 '_id': ObjectId('60aa1eb69343697468b4ce0c'),
 'air_temperature_celcius': 16,
 'date': '24-12-2018',
 'max_wind_speed': 16.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 47.0,
 'windspeed_knots': 12.0}
{'GHI_w/m2': 145,
 '_id': ObjectId('60aa1eb69343697468b4ce0d'),
 'air_temperature_celcius': 17,
 'date': '25-12-2018',
 'max_wind_speed': 16.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 50.4,
 'windspeed_k

In [13]:
climate_date.count()

366

From the count above, we can see that every date from the csv file has been sucessfully inserted into the document.

# Creation of collection for hotspot

In [14]:
# Drop the collection before creating a new collection to prevent
# double insertion
db.hotspot_col.drop()
hotspot_col = db.hotspot_col

I will now perform a simple tuple checking on (lat, long) to determine if the data has duplicate. This process is required to perform insertion to document

In [15]:
mem = []
counter = 0
for i in range(len(hotspot)):
    lat = hotspot.latitude[i]
    long = hotspot.longitude[i]
    comb = (lat, long)
    if comb not in mem:
        mem.append(comb)
    else:
        print("Found duplicate")
        counter += 1
        print(comb)
print("The number of repetition of (lat, long) is " + str(counter))

Found duplicate
(-36.397, 142.254)
Found duplicate
(-37.067, 141.379)
Found duplicate
(-37.333, 143.534)
Found duplicate
(-37.885999999999996, 147.207)
Found duplicate
(-37.062, 141.373)
Found duplicate
(-36.779, 146.108)
Found duplicate
(-36.834, 142.524)
Found duplicate
(-37.856, 143.416)
Found duplicate
(-37.858000000000004, 143.428)
The number of repetition of (lat, long) is 9


From the results above, it seems that there are indeed duplicate values of (lat, long) where the fields are similar. Hence, it should not be inserted into the collection.

In [16]:
for i in range(len(hotspot)):
    # If latitude and longitude already existed, do not insert
    if hotspot_col.find({"$and":[{hotspot.columns[0]:float(hotspot[hotspot.columns[0]][i])}, {hotspot.columns[1]:float(hotspot[hotspot.columns[1]][i])}]}).count() > 0: 
        continue
    else:
        # Else, insert a new latitude and longitude with its corresponding report
        temp = hotspot[hotspot.columns[4]][i].replace("/", "-")
        record = {
            hotspot.columns[0]:float(hotspot[hotspot.columns[0]][i]),
            hotspot.columns[1]:float(hotspot[hotspot.columns[1]][i]),
            "reports":[
                {
                    hotspot.columns[2]:str(hotspot[hotspot.columns[2]][i]),
                    hotspot.columns[3]:int(hotspot[hotspot.columns[3]][i]),
                    hotspot.columns[4]:str(temp),
                    hotspot.columns[5]:int(hotspot[hotspot.columns[5]][i])
                }
            ]
        }
        hotspot_col.insert_one(record)

In [17]:
# Check the document of station
from pprint import pprint

cursor = hotspot_col.find({})
for document in cursor: 
    pprint(document)

{'_id': ObjectId('60aa1eb79343697468b4ce15'),
 'latitude': -37.966,
 'longitude': 145.05100000000002,
 'reports': [{'confidence': 78,
              'date': '27-12-2018',
              'datetime': '2018-12-27T04:16:51',
              'surface_temperature_celcius': 68}]}
{'_id': ObjectId('60aa1eb79343697468b4ce16'),
 'latitude': -35.541,
 'longitude': 143.311,
 'reports': [{'confidence': 82,
              'date': '27-12-2018',
              'datetime': '2018-12-27T00:02:15',
              'surface_temperature_celcius': 63}]}
{'_id': ObjectId('60aa1eb79343697468b4ce17'),
 'latitude': -35.554,
 'longitude': 143.30700000000002,
 'reports': [{'confidence': 67,
              'date': '27-12-2018',
              'datetime': '2018-12-27T00:02:15',
              'surface_temperature_celcius': 53}]}
{'_id': ObjectId('60aa1eb79343697468b4ce18'),
 'latitude': -35.543,
 'longitude': 143.316,
 'reports': [{'confidence': 86,
              'date': '27-12-2018',
              'datetime': '2018-12-27T00:0

              'surface_temperature_celcius': 38}]}
{'_id': ObjectId('60aa1eb89343697468b4ce74'),
 'latitude': -38.16,
 'longitude': 143.803,
 'reports': [{'confidence': 62,
              'date': '13-11-2018',
              'datetime': '2018-11-13T01:16:37',
              'surface_temperature_celcius': 41}]}
{'_id': ObjectId('60aa1eb89343697468b4ce75'),
 'latitude': -36.942,
 'longitude': 143.282,
 'reports': [{'confidence': 63,
              'date': '12-11-2018',
              'datetime': '2018-11-12T04:47:32',
              'surface_temperature_celcius': 53}]}
{'_id': ObjectId('60aa1eb89343697468b4ce76'),
 'latitude': -37.864000000000004,
 'longitude': 144.174,
 'reports': [{'confidence': 58,
              'date': '12-11-2018',
              'datetime': '2018-11-12T04:47:16',
              'surface_temperature_celcius': 41}]}
{'_id': ObjectId('60aa1eb89343697468b4ce77'),
 'latitude': -38.912,
 'longitude': 146.27200000000002,
 'reports': [{'confidence': 56,
              'date': '12-1

              'surface_temperature_celcius': 48}]}
{'_id': ObjectId('60aa1eb89343697468b4cec5'),
 'latitude': -37.375,
 'longitude': 148.054,
 'reports': [{'confidence': 82,
              'date': '3-10-2018',
              'datetime': '2018-10-03T03:58:15',
              'surface_temperature_celcius': 49}]}
{'_id': ObjectId('60aa1eb89343697468b4cec6'),
 'latitude': -37.385999999999996,
 'longitude': 148.043,
 'reports': [{'confidence': 84,
              'date': '3-10-2018',
              'datetime': '2018-10-03T03:58:15',
              'surface_temperature_celcius': 55}]}
{'_id': ObjectId('60aa1eb89343697468b4cec7'),
 'latitude': -37.384,
 'longitude': 148.056,
 'reports': [{'confidence': 90,
              'date': '3-10-2018',
              'datetime': '2018-10-03T03:58:15',
              'surface_temperature_celcius': 63}]}
{'_id': ObjectId('60aa1eb89343697468b4cec8'),
 'latitude': -37.367,
 'longitude': 148.04,
 'reports': [{'confidence': 71,
              'date': '3-10-2018',
      

              'surface_temperature_celcius': 50}]}
{'_id': ObjectId('60aa1eb89343697468b4cf16'),
 'latitude': -37.586,
 'longitude': 148.03799999999998,
 'reports': [{'confidence': 77,
              'date': '23-9-2018',
              'datetime': '2018-09-23T04:59:10',
              'surface_temperature_celcius': 47}]}
{'_id': ObjectId('60aa1eb89343697468b4cf17'),
 'latitude': -36.996,
 'longitude': 148.22799999999998,
 'reports': [{'confidence': 70,
              'date': '23-9-2018',
              'datetime': '2018-09-23T03:21:28',
              'surface_temperature_celcius': 40}]}
{'_id': ObjectId('60aa1eb89343697468b4cf18'),
 'latitude': -36.995,
 'longitude': 148.268,
 'reports': [{'confidence': 56,
              'date': '23-9-2018',
              'datetime': '2018-09-23T03:21:28',
              'surface_temperature_celcius': 37}]}
{'_id': ObjectId('60aa1eb89343697468b4cf19'),
 'latitude': -37.554,
 'longitude': 148.07299999999998,
 'reports': [{'confidence': 63,
              'date

              'surface_temperature_celcius': 61}]}
{'_id': ObjectId('60aa1eb89343697468b4cf67'),
 'latitude': -36.2111,
 'longitude': 141.505,
 'reports': [{'confidence': 68,
              'date': '7-6-2018',
              'datetime': '2018-06-07T04:35:40',
              'surface_temperature_celcius': 44}]}
{'_id': ObjectId('60aa1eb89343697468b4cf68'),
 'latitude': -36.5794,
 'longitude': 142.5959,
 'reports': [{'confidence': 53,
              'date': '7-6-2018',
              'datetime': '2018-06-07T04:35:30',
              'surface_temperature_celcius': 39}]}
{'_id': ObjectId('60aa1eb89343697468b4cf69'),
 'latitude': -36.1002,
 'longitude': 142.3405,
 'reports': [{'confidence': 96,
              'date': '7-6-2018',
              'datetime': '2018-06-07T04:33:50',
              'surface_temperature_celcius': 78}]}
{'_id': ObjectId('60aa1eb89343697468b4cf6a'),
 'latitude': -36.3096,
 'longitude': 142.7722,
 'reports': [{'confidence': 85,
              'date': '7-6-2018',
              

...

 'latitude': -36.0994,
 'longitude': 143.4156,
 'reports': [{'confidence': 82,
              'date': '4-4-2018',
              'datetime': '2018-04-04T04:33:10',
              'surface_temperature_celcius': 62}]}
{'_id': ObjectId('60aa1ebc9343697468b4d74e'),
 'latitude': -36.8395,
 'longitude': 142.2188,
 'reports': [{'confidence': 82,
              'date': '4-4-2018',
              'datetime': '2018-04-04T04:33:10',
              'surface_temperature_celcius': 62}]}
{'_id': ObjectId('60aa1ebc9343697468b4d74f'),
 'latitude': -36.9294,
 'longitude': 142.7087,
 'reports': [{'confidence': 78,
              'date': '4-4-2018',
              'datetime': '2018-04-04T04:33:10',
              'surface_temperature_celcius': 51}]}
{'_id': ObjectId('60aa1ebc9343697468b4d750'),
 'latitude': -36.1439,
 'longitude': 145.832,
 'reports': [{'confidence': 53,
              'date': '4-4-2018',
              'datetime': '2018-04-04T04:33:10',
              'surface_temperature_celcius': 50}]}
{'_id': Obj

 'reports': [{'confidence': 66,
              'date': '3-4-2018',
              'datetime': '2018-04-03T03:52:10',
              'surface_temperature_celcius': 43}]}
{'_id': ObjectId('60aa1ebc9343697468b4d792'),
 'latitude': -37.3851,
 'longitude': 145.8878,
 'reports': [{'confidence': 83,
              'date': '3-4-2018',
              'datetime': '2018-04-03T03:51:30',
              'surface_temperature_celcius': 63}]}
{'_id': ObjectId('60aa1ebc9343697468b4d793'),
 'latitude': -37.7446,
 'longitude': 143.4085,
 'reports': [{'confidence': 89,
              'date': '3-4-2018',
              'datetime': '2018-04-03T03:51:30',
              'surface_temperature_celcius': 66}]}
{'_id': ObjectId('60aa1ebc9343697468b4d794'),
 'latitude': -36.7159,
 'longitude': 141.3991,
 'reports': [{'confidence': 75,
              'date': '3-4-2018',
              'datetime': '2018-04-03T03:51:10',
              'surface_temperature_celcius': 49}]}
{'_id': ObjectId('60aa1ebc9343697468b4d795'),
 'latitude'

 'longitude': 142.72799999999998,
 'reports': [{'confidence': 93,
              'date': '28-3-2018',
              'datetime': '2018-03-28T04:32:20',
              'surface_temperature_celcius': 73}]}
{'_id': ObjectId('60aa1ebc9343697468b4d7e3'),
 'latitude': -36.8202,
 'longitude': 141.9362,
 'reports': [{'confidence': 76,
              'date': '28-3-2018',
              'datetime': '2018-03-28T04:31:50',
              'surface_temperature_celcius': 50}]}
{'_id': ObjectId('60aa1ebc9343697468b4d7e4'),
 'latitude': -36.9576,
 'longitude': 141.5387,
 'reports': [{'confidence': 70,
              'date': '28-3-2018',
              'datetime': '2018-03-28T04:31:50',
              'surface_temperature_celcius': 45}]}
{'_id': ObjectId('60aa1ebc9343697468b4d7e5'),
 'latitude': -36.811,
 'longitude': 142.2655,
 'reports': [{'confidence': 83,
              'date': '28-3-2018',
              'datetime': '2018-03-28T04:30:50',
              'surface_temperature_celcius': 56}]}
{'_id': ObjectId('60

              'surface_temperature_celcius': 75}]}
{'_id': ObjectId('60aa1ebc9343697468b4d829'),
 'latitude': -37.5629,
 'longitude': 144.752,
 'reports': [{'confidence': 85,
              'date': '25-3-2018',
              'datetime': '2018-03-25T03:57:10',
              'surface_temperature_celcius': 59}]}
{'_id': ObjectId('60aa1ebd9343697468b4d82a'),
 'latitude': -37.8122,
 'longitude': 146.0807,
 'reports': [{'confidence': 100,
              'date': '25-3-2018',
              'datetime': '2018-03-25T03:57:10',
              'surface_temperature_celcius': 111}]}
{'_id': ObjectId('60aa1ebd9343697468b4d82b'),
 'latitude': -37.8774,
 'longitude': 145.8213,
 'reports': [{'confidence': 95,
              'date': '25-3-2018',
              'datetime': '2018-03-25T03:57:10',
              'surface_temperature_celcius': 77}]}
{'_id': ObjectId('60aa1ebd9343697468b4d82c'),
 'latitude': -37.5742,
 'longitude': 144.7536,
 'reports': [{'confidence': 75,
              'date': '25-3-2018',
        

              'surface_temperature_celcius': 68}]}
{'_id': ObjectId('60aa1ebd9343697468b4d867'),
 'latitude': -37.4128,
 'longitude': 147.0242,
 'reports': [{'confidence': 85,
              'date': '12-3-2018',
              'datetime': '2018-03-12T04:27:20',
              'surface_temperature_celcius': 98}]}
{'_id': ObjectId('60aa1ebd9343697468b4d868'),
 'latitude': -37.223,
 'longitude': 147.9431,
 'reports': [{'confidence': 100,
              'date': '10-3-2018',
              'datetime': '2018-03-10T04:48:40',
              'surface_temperature_celcius': 105}]}
{'_id': ObjectId('60aa1ebd9343697468b4d869'),
 'latitude': -37.2252,
 'longitude': 147.9363,
 'reports': [{'confidence': 100,
              'date': '10-3-2018',
              'datetime': '2018-03-10T04:46:20',
              'surface_temperature_celcius': 109}]}
{'_id': ObjectId('60aa1ebd9343697468b4d86a'),
 'latitude': -37.2284,
 'longitude': 147.9187,
 'reports': [{'confidence': 94,
              'date': '10-3-2018',
      

In [18]:
print(cursor.count())

2659


In [19]:
len(hotspot)

2668

As we can observe from a cell and 2 cells above (2668, 2659), the difference between the values is 9, which equates to the number of duplicated values of (lat, long). Hence, we are sure that all the document have been successfully inserted into the collection and there are no duplicates.

# Task 2 Question 2

2. Write queries to answer the following tasks on fit3182_assignment_db and
corresponding collection(s). You need to write the queries as a python program using
the pymongo library in Jupyter Notebook. <br>
a. Find climate data on 12th December 2018. <br>
b. Find the latitude, longitude, surface temperature (°C), and confidence when
the surface temperature (°C) was between 65 °C and 100 °C. <br>
c. Find date, surface temperature (°C), air temperature (°C), relative humidity
and max wind speed on 15th and 16th of December 2018. <br>
d. Find datetime, air temperature (°C), surface temperature (°C) and
confidence when the confidence is between 80 and 100. <br>
e. Find the top 10 records with the highest surface temperature (°C). <br>
f. Find the number of fires each day. You are required to only display the total
number of fires and the date in the output. <br>
g. Find the records of fires where the confidence is below 70. <br>
h. Find the average surface temperature (°C) for each day. You are required to
only display average surface temperature (°C) and the date in the output. <br>
i. Find the top 10 records with the lowest GHI. <br>
j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35. <br>


a.

In [20]:
cursor = climate_date.aggregate([
    {
        "$lookup":
        {
            "from":"climate_station",
            "localField":"date",
            "foreignField":"date",
            "as": "station"
        }
    }
    ,{"$match": {"date":"12-12-2018"}}
    ,{"$project": {"station.date":0, "_id":0}}
])

In [21]:
for document in cursor:
    pprint(document)

{'GHI_w/m2': 156,
 'air_temperature_celcius': 19,
 'date': '12-12-2018',
 'max_wind_speed': 12.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 55.3,
 'station': [{'_id': ObjectId('60aa1eb59343697468b4cca6'), 'station': 948702}],
 'windspeed_knots': 6.2}


b.

In [22]:
cursor = hotspot_col.find({
    "reports.surface_temperature_celcius":{"$gte": 65, "$lte":100}, 
    }, {"reports.date":0, "reports.datetime":0})

In [23]:
for document in cursor:
    pprint(document)

{'_id': ObjectId('60aa1eb79343697468b4ce15'),
 'latitude': -37.966,
 'longitude': 145.05100000000002,
 'reports': [{'confidence': 78, 'surface_temperature_celcius': 68}]}
{'_id': ObjectId('60aa1eb79343697468b4ce18'),
 'latitude': -35.543,
 'longitude': 143.316,
 'reports': [{'confidence': 86, 'surface_temperature_celcius': 67}]}
{'_id': ObjectId('60aa1eb79343697468b4ce1f'),
 'latitude': -37.875,
 'longitude': 142.51,
 'reports': [{'confidence': 93, 'surface_temperature_celcius': 73}]}
{'_id': ObjectId('60aa1eb79343697468b4ce21'),
 'latitude': -37.613,
 'longitude': 149.305,
 'reports': [{'confidence': 95, 'surface_temperature_celcius': 75}]}
{'_id': ObjectId('60aa1eb79343697468b4ce23'),
 'latitude': -37.624,
 'longitude': 149.314,
 'reports': [{'confidence': 90, 'surface_temperature_celcius': 66}]}
{'_id': ObjectId('60aa1eb79343697468b4ce26'),
 'latitude': -38.056999999999995,
 'longitude': 144.211,
 'reports': [{'confidence': 93, 'surface_temperature_celcius': 73}]}
{'_id': ObjectId('

 'longitude': 146.6642,
 'reports': [{'confidence': 91, 'surface_temperature_celcius': 68}]}
{'_id': ObjectId('60aa1eb89343697468b4cfa9'),
 'latitude': -36.8359,
 'longitude': 146.6448,
 'reports': [{'confidence': 96, 'surface_temperature_celcius': 78}]}
{'_id': ObjectId('60aa1eb89343697468b4cfac'),
 'latitude': -37.9085,
 'longitude': 141.1038,
 'reports': [{'confidence': 98, 'surface_temperature_celcius': 82}]}
{'_id': ObjectId('60aa1eb89343697468b4cfaf'),
 'latitude': -38.4349,
 'longitude': 146.3122,
 'reports': [{'confidence': 100, 'surface_temperature_celcius': 93}]}
{'_id': ObjectId('60aa1eb89343697468b4cfb0'),
 'latitude': -38.4792,
 'longitude': 146.3081,
 'reports': [{'confidence': 89, 'surface_temperature_celcius': 65}]}
{'_id': ObjectId('60aa1eb89343697468b4cfb5'),
 'latitude': -37.9047,
 'longitude': 141.0945,
 'reports': [{'confidence': 100, 'surface_temperature_celcius': 90}]}
{'_id': ObjectId('60aa1eb89343697468b4cfc5'),
 'latitude': -37.4984,
 'longitude': 142.9739,
 '

 'longitude': 147.2691,
 'reports': [{'confidence': 96, 'surface_temperature_celcius': 78}]}
{'_id': ObjectId('60aa1eb99343697468b4d1cd'),
 'latitude': -36.5454,
 'longitude': 144.7402,
 'reports': [{'confidence': 100, 'surface_temperature_celcius': 96}]}
{'_id': ObjectId('60aa1eb99343697468b4d1d1'),
 'latitude': -36.7593,
 'longitude': 142.895,
 'reports': [{'confidence': 92, 'surface_temperature_celcius': 69}]}
{'_id': ObjectId('60aa1eb99343697468b4d1db'),
 'latitude': -36.3435,
 'longitude': 141.5513,
 'reports': [{'confidence': 90, 'surface_temperature_celcius': 67}]}
{'_id': ObjectId('60aa1eb99343697468b4d1e1'),
 'latitude': -36.6942,
 'longitude': 143.8021,
 'reports': [{'confidence': 93, 'surface_temperature_celcius': 72}]}
{'_id': ObjectId('60aa1eb99343697468b4d1e4'),
 'latitude': -36.3459,
 'longitude': 141.5586,
 'reports': [{'confidence': 100, 'surface_temperature_celcius': 91}]}
{'_id': ObjectId('60aa1eb99343697468b4d1e9'),
 'latitude': -36.4046,
 'longitude': 141.1534,
 'r

 'latitude': -36.7101,
 'longitude': 142.981,
 'reports': [{'confidence': 77, 'surface_temperature_celcius': 68}]}
{'_id': ObjectId('60aa1eb99343697468b4d300'),
 'latitude': -37.0959,
 'longitude': 143.8206,
 'reports': [{'confidence': 89, 'surface_temperature_celcius': 65}]}
{'_id': ObjectId('60aa1eb99343697468b4d304'),
 'latitude': -37.9594,
 'longitude': 143.5638,
 'reports': [{'confidence': 100, 'surface_temperature_celcius': 87}]}
{'_id': ObjectId('60aa1eb99343697468b4d30a'),
 'latitude': -36.5921,
 'longitude': 144.4905,
 'reports': [{'confidence': 98, 'surface_temperature_celcius': 82}]}
{'_id': ObjectId('60aa1eb99343697468b4d316'),
 'latitude': -36.6787,
 'longitude': 144.8042,
 'reports': [{'confidence': 96, 'surface_temperature_celcius': 79}]}
{'_id': ObjectId('60aa1eb99343697468b4d31e'),
 'latitude': -38.0423,
 'longitude': 146.4048,
 'reports': [{'confidence': 84, 'surface_temperature_celcius': 66}]}
{'_id': ObjectId('60aa1eb99343697468b4d327'),
 'latitude': -38.1224,
 'lon

 'longitude': 144.763,
 'reports': [{'confidence': 97, 'surface_temperature_celcius': 81}]}
{'_id': ObjectId('60aa1ebb9343697468b4d54c'),
 'latitude': -36.4612,
 'longitude': 144.7775,
 'reports': [{'confidence': 91, 'surface_temperature_celcius': 80}]}
{'_id': ObjectId('60aa1ebb9343697468b4d54f'),
 'latitude': -36.7089,
 'longitude': 143.3518,
 'reports': [{'confidence': 100, 'surface_temperature_celcius': 93}]}
{'_id': ObjectId('60aa1ebb9343697468b4d55a'),
 'latitude': -36.601,
 'longitude': 144.6378,
 'reports': [{'confidence': 97, 'surface_temperature_celcius': 80}]}
{'_id': ObjectId('60aa1ebb9343697468b4d563'),
 'latitude': -36.8681,
 'longitude': 143.5186,
 'reports': [{'confidence': 95, 'surface_temperature_celcius': 76}]}
{'_id': ObjectId('60aa1ebb9343697468b4d56d'),
 'latitude': -37.1999,
 'longitude': 143.8231,
 'reports': [{'confidence': 100, 'surface_temperature_celcius': 90}]}
{'_id': ObjectId('60aa1ebb9343697468b4d579'),
 'latitude': -37.6171,
 'longitude': 142.9426,
 're

 'latitude': -37.8161,
 'longitude': 142.9286,
 'reports': [{'confidence': 92, 'surface_temperature_celcius': 71}]}
{'_id': ObjectId('60aa1ebc9343697468b4d6cd'),
 'latitude': -36.0387,
 'longitude': 145.6787,
 'reports': [{'confidence': 90, 'surface_temperature_celcius': 67}]}
{'_id': ObjectId('60aa1ebc9343697468b4d6cf'),
 'latitude': -36.6349,
 'longitude': 142.2125,
 'reports': [{'confidence': 94, 'surface_temperature_celcius': 86}]}
{'_id': ObjectId('60aa1ebc9343697468b4d6d5'),
 'latitude': -36.9018,
 'longitude': 141.0146,
 'reports': [{'confidence': 100, 'surface_temperature_celcius': 100}]}
{'_id': ObjectId('60aa1ebc9343697468b4d6d6'),
 'latitude': -36.6986,
 'longitude': 142.7259,
 'reports': [{'confidence': 86, 'surface_temperature_celcius': 72}]}
{'_id': ObjectId('60aa1ebc9343697468b4d6e1'),
 'latitude': -38.0052,
 'longitude': 142.6179,
 'reports': [{'confidence': 94, 'surface_temperature_celcius': 89}]}
{'_id': ObjectId('60aa1ebc9343697468b4d6f7'),
 'latitude': -37.6387,
 'l

 'longitude': 147.9187,
 'reports': [{'confidence': 94, 'surface_temperature_celcius': 73}]}
{'_id': ObjectId('60aa1ebd9343697468b4d86b'),
 'latitude': -37.6572,
 'longitude': 142.0703,
 'reports': [{'confidence': 97, 'surface_temperature_celcius': 80}]}


c.

In [24]:
cursor = climate_date.aggregate([
    {
        "$lookup":
        {
            "from":"hotspot_col",
            "localField":"date",
            "foreignField":"reports.date",
            "as":"joined_hotspot"
        }
    }
    ,{"$unwind":"$joined_hotspot"}
    ,{"$unwind":"$joined_hotspot.reports"}
    ,{"$match":{"$or": [{"date": "15-12-2018"}, {"date":"16-12-2018"}]}}
    ,{"$project":{
        "windspeed_knots":0, 
        "precipitation ":0,
        "GHI_w/m2":0,
        "joined_hotspot.latitude":0,
        "joined_hotspot.longitude":0,
        "joined_hotspot._id":0,
        "joined_hotspot.reports.confidence":0,
        "joined_hotspot.reports.datetime":0
        }
     }
])

In [25]:
for document in cursor:
    pprint(document)

{'_id': ObjectId('60aa1eb69343697468b4ce03'),
 'air_temperature_celcius': 18,
 'date': '15-12-2018',
 'joined_hotspot': {'reports': {'date': '15-12-2018',
                                'surface_temperature_celcius': 42}},
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'_id': ObjectId('60aa1eb69343697468b4ce03'),
 'air_temperature_celcius': 18,
 'date': '15-12-2018',
 'joined_hotspot': {'reports': {'date': '15-12-2018',
                                'surface_temperature_celcius': 36}},
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'_id': ObjectId('60aa1eb69343697468b4ce03'),
 'air_temperature_celcius': 18,
 'date': '15-12-2018',
 'joined_hotspot': {'reports': {'date': '15-12-2018',
                                'surface_temperature_celcius': 38}},
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'_id': ObjectId('60aa1eb69343697468b4ce03'),
 'air_temperature_celcius': 18,
 'date': '15-12-2018',
 'joined_hotspot': {'reports': {'date': '15-12-2018',
              

d.

In [26]:
cursor = hotspot_col.aggregate([
    {
        "$lookup":{
            "from":"climate_date",
            "localField":"reports.date",
            "foreignField":"date",
            "as":"join_climate"}
    },
    {"$unwind": "$reports"},
    {"$match": {
        "$and": [
            {"reports.confidence": {"$gte": 80}},
            {"reports.confidence": {"$lte": 100}}
        ]
    }},
    {"$project":{
        "latitude":0,
        "longitude":0,
        "join_climate.relative_humidity":0,
        "join_climate.precipitation ":0,
        "join_climate.max_wind_speed":0,
        "join_climate.windspeed_knots":0,
        "join_climate.GHI_w/m2":0,
        "join_climate.date":0,
        "reports.date":0
    }}
])

In [27]:
for document in cursor:
    pprint(document)

{'_id': ObjectId('60aa1eb79343697468b4ce16'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4ce0f'),
                   'air_temperature_celcius': 28}],
 'reports': {'confidence': 82,
             'datetime': '2018-12-27T00:02:15',
             'surface_temperature_celcius': 63}}
{'_id': ObjectId('60aa1eb79343697468b4ce18'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4ce0f'),
                   'air_temperature_celcius': 28}],
 'reports': {'confidence': 86,
             'datetime': '2018-12-27T00:02:14',
             'surface_temperature_celcius': 67}}
{'_id': ObjectId('60aa1eb79343697468b4ce19'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4ce0d'),
                   'air_temperature_celcius': 17}],
 'reports': {'confidence': 80,
             'datetime': '2018-12-25T04:29:08',
             'surface_temperature_celcius': 54}}
{'_id': ObjectId('60aa1eb79343697468b4ce1c'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4ce04'),
                   '

{'_id': ObjectId('60aa1eb89343697468b4cea4'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cdc9'),
                   'air_temperature_celcius': 26}],
 'reports': {'confidence': 84,
             'datetime': '2018-10-18T00:39:02',
             'surface_temperature_celcius': 59}}
{'_id': ObjectId('60aa1eb89343697468b4cea5'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cdc9'),
                   'air_temperature_celcius': 26}],
 'reports': {'confidence': 82,
             'datetime': '2018-10-18T00:39:02',
             'surface_temperature_celcius': 56}}
{'_id': ObjectId('60aa1eb89343697468b4cea7'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cdc8'),
                   'air_temperature_celcius': 22}],
 'reports': {'confidence': 94,
             'datetime': '2018-10-17T13:36:12',
             'surface_temperature_celcius': 43}}
{'_id': ObjectId('60aa1eb89343697468b4cea9'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cdc8'),
                   '

{'_id': ObjectId('60aa1eb89343697468b4ceea'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cdb1'),
                   'air_temperature_celcius': 14}],
 'reports': {'confidence': 100,
             'datetime': '2018-09-24T15:07:47',
             'surface_temperature_celcius': 65}}
{'_id': ObjectId('60aa1eb89343697468b4ceed'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cdb1'),
                   'air_temperature_celcius': 14}],
 'reports': {'confidence': 90,
             'datetime': '2018-09-24T15:07:47',
             'surface_temperature_celcius': 41}}
{'_id': ObjectId('60aa1eb89343697468b4ceee'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cdb1'),
                   'air_temperature_celcius': 14}],
 'reports': {'confidence': 99,
             'datetime': '2018-09-24T15:07:47',
             'surface_temperature_celcius': 61}}
{'_id': ObjectId('60aa1eb89343697468b4cef1'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cdb1'),
                   

 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd2d'),
                   'air_temperature_celcius': 10}],
 'reports': {'confidence': 94,
             'datetime': '2018-05-15T04:26:20',
             'surface_temperature_celcius': 74}}
{'_id': ObjectId('60aa1eb89343697468b4d015'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd2d'),
                   'air_temperature_celcius': 10}],
 'reports': {'confidence': 100,
             'datetime': '2018-05-15T04:26:20',
             'surface_temperature_celcius': 110}}
{'_id': ObjectId('60aa1eb89343697468b4d017'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd2d'),
                   'air_temperature_celcius': 10}],
 'reports': {'confidence': 89,
             'datetime': '2018-05-15T04:26:20',
             'surface_temperature_celcius': 65}}
{'_id': ObjectId('60aa1eb89343697468b4d01a'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd2d'),
                   'air_temperature_celcius': 10}],
 'reports': 

 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd28'),
                   'air_temperature_celcius': 10}],
 'reports': {'confidence': 81,
             'datetime': '2018-05-10T04:08:10',
             'surface_temperature_celcius': 54}}
{'_id': ObjectId('60aa1eb89343697468b4d0d2'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd28'),
                   'air_temperature_celcius': 10}],
 'reports': {'confidence': 96,
             'datetime': '2018-05-10T04:08:10',
             'surface_temperature_celcius': 78}}
{'_id': ObjectId('60aa1eb89343697468b4d0d6'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd28'),
                   'air_temperature_celcius': 10}],
 'reports': {'confidence': 87,
             'datetime': '2018-05-10T04:08:10',
             'surface_temperature_celcius': 62}}
{'_id': ObjectId('60aa1eb89343697468b4d0d7'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd28'),
                   'air_temperature_celcius': 10}],
 'reports': {'

...

{'_id': ObjectId('60aa1ebb9343697468b4d5bd'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd0d'),
                   'air_temperature_celcius': 16}],
 'reports': {'confidence': 100,
             'datetime': '2018-04-13T04:26:30',
             'surface_temperature_celcius': 92}}
{'_id': ObjectId('60aa1ebb9343697468b4d5be'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd0d'),
                   'air_temperature_celcius': 16}],
 'reports': {'confidence': 83,
             'datetime': '2018-04-13T04:26:30',
             'surface_temperature_celcius': 56}}
{'_id': ObjectId('60aa1ebb9343697468b4d5bf'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd0d'),
                   'air_temperature_celcius': 16}],
 'reports': {'confidence': 100,
             'datetime': '2018-04-13T04:26:30',
             'surface_temperature_celcius': 111}}
{'_id': ObjectId('60aa1ebb9343697468b4d5c0'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd0d'),
                 

{'_id': ObjectId('60aa1ebb9343697468b4d658'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd07'),
                   'air_temperature_celcius': 21}],
 'reports': {'confidence': 92,
             'datetime': '2018-04-07T12:50:50',
             'surface_temperature_celcius': 43}}
{'_id': ObjectId('60aa1ebb9343697468b4d65e'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd07'),
                   'air_temperature_celcius': 21}],
 'reports': {'confidence': 89,
             'datetime': '2018-04-07T12:50:40',
             'surface_temperature_celcius': 42}}
{'_id': ObjectId('60aa1ebb9343697468b4d65f'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd07'),
                   'air_temperature_celcius': 21}],
 'reports': {'confidence': 80,
             'datetime': '2018-04-07T05:09:50',
             'surface_temperature_celcius': 54}}
{'_id': ObjectId('60aa1ebb9343697468b4d663'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd07'),
                   '

 'reports': {'confidence': 100,
             'datetime': '2018-04-05T13:03:20',
             'surface_temperature_celcius': 55}}
{'_id': ObjectId('60aa1ebc9343697468b4d6f0'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd05'),
                   'air_temperature_celcius': 17}],
 'reports': {'confidence': 98,
             'datetime': '2018-04-05T13:03:00',
             'surface_temperature_celcius': 45}}
{'_id': ObjectId('60aa1ebc9343697468b4d6f1'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd05'),
                   'air_temperature_celcius': 17}],
 'reports': {'confidence': 100,
             'datetime': '2018-04-05T13:03:00',
             'surface_temperature_celcius': 55}}
{'_id': ObjectId('60aa1ebc9343697468b4d6f2'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd05'),
                   'air_temperature_celcius': 17}],
 'reports': {'confidence': 81,
             'datetime': '2018-04-05T13:03:00',
             'surface_temperature_celcius': 39}}


 'reports': {'confidence': 100,
             'datetime': '2018-04-03T03:52:30',
             'surface_temperature_celcius': 90}}
{'_id': ObjectId('60aa1ebc9343697468b4d789'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd03'),
                   'air_temperature_celcius': 15}],
 'reports': {'confidence': 100,
             'datetime': '2018-04-03T03:52:20',
             'surface_temperature_celcius': 115}}
{'_id': ObjectId('60aa1ebc9343697468b4d78b'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd03'),
                   'air_temperature_celcius': 15}],
 'reports': {'confidence': 86,
             'datetime': '2018-04-03T03:52:10',
             'surface_temperature_celcius': 61}}
{'_id': ObjectId('60aa1ebc9343697468b4d78d'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4cd03'),
                   'air_temperature_celcius': 15}],
 'reports': {'confidence': 100,
             'datetime': '2018-04-03T03:52:10',
             'surface_temperature_celcius': 98}

 'reports': {'confidence': 98,
             'datetime': '2018-03-19T04:33:00',
             'surface_temperature_celcius': 83}}
{'_id': ObjectId('60aa1ebd9343697468b4d83e'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4ccf4'),
                   'air_temperature_celcius': 24}],
 'reports': {'confidence': 100,
             'datetime': '2018-03-19T04:32:50',
             'surface_temperature_celcius': 91}}
{'_id': ObjectId('60aa1ebd9343697468b4d83f'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4ccf4'),
                   'air_temperature_celcius': 24}],
 'reports': {'confidence': 84,
             'datetime': '2018-03-19T04:32:50',
             'surface_temperature_celcius': 58}}
{'_id': ObjectId('60aa1ebd9343697468b4d841'),
 'join_climate': [{'_id': ObjectId('60aa1eb69343697468b4ccf4'),
                   'air_temperature_celcius': 24}],
 'reports': {'confidence': 100,
             'datetime': '2018-03-19T04:32:40',
             'surface_temperature_celcius': 113}}

e.

In [28]:
cursor = hotspot_col.aggregate([
    {"$unwind":"$reports"},
    {"$sort": {"reports.surface_temperature_celcius": -1}},
    {"$limit": 10}
])

In [29]:
for document in cursor:
    pprint(document)

{'_id': ObjectId('60aa1eb99343697468b4d2af'),
 'latitude': -38.1665,
 'longitude': 143.062,
 'reports': {'confidence': 100,
             'date': '18-4-2018',
             'datetime': '2018-04-18T04:52:00',
             'surface_temperature_celcius': 124}}
{'_id': ObjectId('60aa1ebc9343697468b4d759'),
 'latitude': -36.343,
 'longitude': 142.1986,
 'reports': {'confidence': 100,
             'date': '4-4-2018',
             'datetime': '2018-04-04T04:32:50',
             'surface_temperature_celcius': 123}}
{'_id': ObjectId('60aa1eb99343697468b4d221'),
 'latitude': -36.9318,
 'longitude': 143.0907,
 'reports': {'confidence': 100,
             'date': '1-5-2018',
             'datetime': '2018-05-01T04:14:20',
             'surface_temperature_celcius': 122}}
{'_id': ObjectId('60aa1ebd9343697468b4d848'),
 'latitude': -37.016999999999996,
 'longitude': 148.1297,
 'reports': {'confidence': 100,
             'date': '18-3-2018',
             'datetime': '2018-03-18T03:50:50',
             's

f.

In [30]:
cursor = climate_date.aggregate([
    {"$lookup": {"from": "hotspot_col",
                 "localField": "date",
                 "foreignField": "reports.date",
                 "as": "merged_date"}},
     {"$project": {"_id":0,
                  "date": 1,
                  "fire count": {"$size": "$merged_date"}}}
])

In [31]:
for document in cursor:
    pprint(document)

{'date': '31-12-2017', 'fire count': 0}
{'date': '2-1-2018', 'fire count': 0}
{'date': '3-1-2018', 'fire count': 0}
{'date': '4-1-2018', 'fire count': 0}
{'date': '5-1-2018', 'fire count': 0}
{'date': '6-1-2018', 'fire count': 0}
{'date': '7-1-2018', 'fire count': 0}
{'date': '8-1-2018', 'fire count': 0}
{'date': '9-1-2018', 'fire count': 0}
{'date': '10-1-2018', 'fire count': 0}
{'date': '11-1-2018', 'fire count': 0}
{'date': '12-1-2018', 'fire count': 0}
{'date': '13-1-2018', 'fire count': 0}
{'date': '14-1-2018', 'fire count': 0}
{'date': '15-1-2018', 'fire count': 0}
{'date': '16-1-2018', 'fire count': 0}
{'date': '17-1-2018', 'fire count': 0}
{'date': '18-1-2018', 'fire count': 0}
{'date': '19-1-2018', 'fire count': 0}
{'date': '20-1-2018', 'fire count': 0}
{'date': '21-1-2018', 'fire count': 0}
{'date': '22-1-2018', 'fire count': 0}
{'date': '23-1-2018', 'fire count': 0}
{'date': '24-1-2018', 'fire count': 0}
{'date': '25-1-2018', 'fire count': 0}
{'date': '26-1-2018', 'fire coun

g.

In [32]:
cursor = hotspot_col.find(
    {"reports.confidence": {"$lt": 70}}
)

In [33]:
for document in cursor:
    pprint(document)

{'_id': ObjectId('60aa1eb79343697468b4ce17'),
 'latitude': -35.554,
 'longitude': 143.30700000000002,
 'reports': [{'confidence': 67,
              'date': '27-12-2018',
              'datetime': '2018-12-27T00:02:15',
              'surface_temperature_celcius': 53}]}
{'_id': ObjectId('60aa1eb79343697468b4ce1a'),
 'latitude': -35.646,
 'longitude': 142.282,
 'reports': [{'confidence': 65,
              'date': '24-12-2018',
              'datetime': '2018-12-24T13:12:01',
              'surface_temperature_celcius': 32}]}
{'_id': ObjectId('60aa1eb79343697468b4ce1d'),
 'latitude': -37.624,
 'longitude': 149.332,
 'reports': [{'confidence': 69,
              'date': '16-12-2018',
              'datetime': '2018-12-16T15:38:39',
              'surface_temperature_celcius': 33}]}
{'_id': ObjectId('60aa1eb79343697468b4ce1e'),
 'latitude': -37.82,
 'longitude': 142.32299999999998,
 'reports': [{'confidence': 62,
              'date': '16-12-2018',
              'datetime': '2018-12-16T04:35

              'surface_temperature_celcius': 41}]}
{'_id': ObjectId('60aa1eb89343697468b4cf07'),
 'latitude': -37.379,
 'longitude': 148.126,
 'reports': [{'confidence': 58,
              'date': '23-9-2018',
              'datetime': '2018-09-23T23:50:40',
              'surface_temperature_celcius': 82}]}
{'_id': ObjectId('60aa1eb89343697468b4cf08'),
 'latitude': -37.379,
 'longitude': 148.132,
 'reports': [{'confidence': 62,
              'date': '23-9-2018',
              'datetime': '2018-09-23T23:50:40',
              'surface_temperature_celcius': 78}]}
{'_id': ObjectId('60aa1eb89343697468b4cf11'),
 'latitude': -36.984,
 'longitude': 148.25,
 'reports': [{'confidence': 50,
              'date': '23-9-2018',
              'datetime': '2018-09-23T04:59:18',
              'surface_temperature_celcius': 35}]}
{'_id': ObjectId('60aa1eb89343697468b4cf13'),
 'latitude': -37.347,
 'longitude': 148.019,
 'reports': [{'confidence': 61,
              'date': '23-9-2018',
              'dat

              'surface_temperature_celcius': 42}]}
{'_id': ObjectId('60aa1eb89343697468b4cfd9'),
 'latitude': -36.2538,
 'longitude': 145.5929,
 'reports': [{'confidence': 63,
              'date': '15-5-2018',
              'datetime': '2018-05-15T04:29:30',
              'surface_temperature_celcius': 42}]}
{'_id': ObjectId('60aa1eb89343697468b4cfda'),
 'latitude': -36.437,
 'longitude': 144.1478,
 'reports': [{'confidence': 66,
              'date': '15-5-2018',
              'datetime': '2018-05-15T04:29:30',
              'surface_temperature_celcius': 43}]}
{'_id': ObjectId('60aa1eb89343697468b4cfdb'),
 'latitude': -36.6217,
 'longitude': 142.0944,
 'reports': [{'confidence': 55,
              'date': '15-5-2018',
              'datetime': '2018-05-15T04:29:10',
              'surface_temperature_celcius': 39}]}
{'_id': ObjectId('60aa1eb89343697468b4cfdc'),
 'latitude': -36.3652,
 'longitude': 141.3508,
 'reports': [{'confidence': 66,
              'date': '15-5-2018',
          

{'_id': ObjectId('60aa1eb89343697468b4d0cb'),
 'latitude': -36.1687,
 'longitude': 145.8275,
 'reports': [{'confidence': 68,
              'date': '10-5-2018',
              'datetime': '2018-05-10T04:08:10',
              'surface_temperature_celcius': 44}]}
{'_id': ObjectId('60aa1eb89343697468b4d0cc'),
 'latitude': -36.3355,
 'longitude': 144.5241,
 'reports': [{'confidence': 51,
              'date': '10-5-2018',
              'datetime': '2018-05-10T04:08:10',
              'surface_temperature_celcius': 38}]}
{'_id': ObjectId('60aa1eb89343697468b4d0ce'),
 'latitude': -36.4944,
 'longitude': 141.8992,
 'reports': [{'confidence': 58,
              'date': '10-5-2018',
              'datetime': '2018-05-10T04:08:10',
              'surface_temperature_celcius': 40}]}
{'_id': ObjectId('60aa1eb89343697468b4d0cf'),
 'latitude': -37.1749,
 'longitude': 143.9573,
 'reports': [{'confidence': 69,
              'date': '10-5-2018',
              'datetime': '2018-05-10T04:08:10',
           

{'_id': ObjectId('60aa1eb99343697468b4d1b9'),
 'latitude': -37.5334,
 'longitude': 143.0587,
 'reports': [{'confidence': 57,
              'date': '4-5-2018',
              'datetime': '2018-05-04T04:44:40',
              'surface_temperature_celcius': 41}]}
{'_id': ObjectId('60aa1eb99343697468b4d1be'),
 'latitude': -36.8261,
 'longitude': 141.4747,
 'reports': [{'confidence': 52,
              'date': '4-5-2018',
              'datetime': '2018-05-04T04:44:40',
              'surface_temperature_celcius': 38}]}
{'_id': ObjectId('60aa1eb99343697468b4d1bf'),
 'latitude': -36.334,
 'longitude': 144.0309,
 'reports': [{'confidence': 62,
              'date': '4-5-2018',
              'datetime': '2018-05-04T04:44:40',
              'surface_temperature_celcius': 41}]}
{'_id': ObjectId('60aa1eb99343697468b4d1c0'),
 'latitude': -36.228,
 'longitude': 144.6721,
 'reports': [{'confidence': 56,
              'date': '4-5-2018',
              'datetime': '2018-05-04T04:44:40',
              'su

{'_id': ObjectId('60aa1eb99343697468b4d2d9'),
 'latitude': -37.6868,
 'longitude': 142.3949,
 'reports': [{'confidence': 59,
              'date': '18-4-2018',
              'datetime': '2018-04-18T04:48:10',
              'surface_temperature_celcius': 40}]}
{'_id': ObjectId('60aa1eb99343697468b4d2dc'),
 'latitude': -36.1021,
 'longitude': 146.6853,
 'reports': [{'confidence': 52,
              'date': '18-4-2018',
              'datetime': '2018-04-18T04:48:10',
              'surface_temperature_celcius': 42}]}
{'_id': ObjectId('60aa1eb99343697468b4d2df'),
 'latitude': -37.6674,
 'longitude': 142.2687,
 'reports': [{'confidence': 65,
              'date': '18-4-2018',
              'datetime': '2018-04-18T04:47:40',
              'surface_temperature_celcius': 42}]}
{'_id': ObjectId('60aa1eb99343697468b4d2e0'),
 'latitude': -37.7181,
 'longitude': 141.6962,
 'reports': [{'confidence': 62,
              'date': '18-4-2018',
              'datetime': '2018-04-18T04:47:20',
           

{'_id': ObjectId('60aa1eba9343697468b4d3b7'),
 'latitude': -36.1029,
 'longitude': 145.0695,
 'reports': [{'confidence': 66,
              'date': '18-4-2018',
              'datetime': '2018-04-18T04:44:50',
              'surface_temperature_celcius': 43}]}
{'_id': ObjectId('60aa1eba9343697468b4d3b8'),
 'latitude': -36.7667,
 'longitude': 141.1177,
 'reports': [{'confidence': 68,
              'date': '18-4-2018',
              'datetime': '2018-04-18T04:44:50',
              'surface_temperature_celcius': 44}]}
{'_id': ObjectId('60aa1eba9343697468b4d3b9'),
 'latitude': -35.9082,
 'longitude': 145.3761,
 'reports': [{'confidence': 69,
              'date': '18-4-2018',
              'datetime': '2018-04-18T04:44:50',
              'surface_temperature_celcius': 44}]}
{'_id': ObjectId('60aa1eba9343697468b4d3ba'),
 'latitude': -36.3115,
 'longitude': 143.3581,
 'reports': [{'confidence': 65,
              'date': '18-4-2018',
              'datetime': '2018-04-18T04:44:50',
           

{'_id': ObjectId('60aa1eba9343697468b4d51a'),
 'latitude': -37.4391,
 'longitude': 143.0293,
 'reports': [{'confidence': 66,
              'date': '13-4-2018',
              'datetime': '2018-04-13T04:26:40',
              'surface_temperature_celcius': 43}]}
{'_id': ObjectId('60aa1eba9343697468b4d51b'),
 'latitude': -37.6161,
 'longitude': 141.2781,
 'reports': [{'confidence': 62,
              'date': '13-4-2018',
              'datetime': '2018-04-13T04:26:40',
              'surface_temperature_celcius': 45}]}
{'_id': ObjectId('60aa1eba9343697468b4d52c'),
 'latitude': -36.6777,
 'longitude': 141.37,
 'reports': [{'confidence': 62,
              'date': '13-4-2018',
              'datetime': '2018-04-13T04:26:30',
              'surface_temperature_celcius': 49}]}
{'_id': ObjectId('60aa1eba9343697468b4d52d'),
 'latitude': -36.6701,
 'longitude': 141.3566,
 'reports': [{'confidence': 61,
              'date': '13-4-2018',
              'datetime': '2018-04-13T04:26:30',
             

 'latitude': -37.34,
 'longitude': 149.3668,
 'reports': [{'confidence': 67,
              'date': '8-4-2018',
              'datetime': '2018-04-08T04:15:20',
              'surface_temperature_celcius': 74}]}
{'_id': ObjectId('60aa1ebb9343697468b4d637'),
 'latitude': -37.8567,
 'longitude': 147.1576,
 'reports': [{'confidence': 62,
              'date': '8-4-2018',
              'datetime': '2018-04-08T04:14:40',
              'surface_temperature_celcius': 48}]}
{'_id': ObjectId('60aa1ebb9343697468b4d63a'),
 'latitude': -37.4368,
 'longitude': 149.1191,
 'reports': [{'confidence': 56,
              'date': '8-4-2018',
              'datetime': '2018-04-08T04:12:20',
              'surface_temperature_celcius': 52}]}
{'_id': ObjectId('60aa1ebb9343697468b4d63e'),
 'latitude': -37.9408,
 'longitude': 146.0611,
 'reports': [{'confidence': 58,
              'date': '8-4-2018',
              'datetime': '2018-04-08T04:11:40',
              'surface_temperature_celcius': 47}]}
{'_id': Obje

 'longitude': 144.7343,
 'reports': [{'confidence': 67,
              'date': '3-4-2018',
              'datetime': '2018-04-03T03:51:10',
              'surface_temperature_celcius': 44}]}
{'_id': ObjectId('60aa1ebc9343697468b4d79c'),
 'latitude': -37.0241,
 'longitude': 141.5163,
 'reports': [{'confidence': 67,
              'date': '3-4-2018',
              'datetime': '2018-04-03T03:50:50',
              'surface_temperature_celcius': 43}]}
{'_id': ObjectId('60aa1ebc9343697468b4d79d'),
 'latitude': -37.2912,
 'longitude': 143.543,
 'reports': [{'confidence': 66,
              'date': '3-4-2018',
              'datetime': '2018-04-03T03:50:50',
              'surface_temperature_celcius': 43}]}
{'_id': ObjectId('60aa1ebc9343697468b4d79e'),
 'latitude': -37.313,
 'longitude': 146.1561,
 'reports': [{'confidence': 52,
              'date': '3-4-2018',
              'datetime': '2018-04-03T03:50:50',
              'surface_temperature_celcius': 46}]}
{'_id': ObjectId('60aa1ebc934369746

h.

In [34]:
cursor = hotspot_col.aggregate([
    {"$unwind":"$reports"},
    {"$group": {"_id": {"Date": "$reports.date"},
                "Average": {"$avg":"$reports.surface_temperature_celcius"}}}
])

In [35]:
for document in cursor:
    pprint(document)

{'Average': 38.5, '_id': {'Date': '13-3-2018'}}
{'Average': 65.6, '_id': {'Date': '14-3-2018'}}
{'Average': 46.0, '_id': {'Date': '15-3-2018'}}
{'Average': 59.5, '_id': {'Date': '17-3-2018'}}
{'Average': 79.33333333333333, '_id': {'Date': '18-3-2018'}}
{'Average': 65.57142857142857, '_id': {'Date': '19-3-2018'}}
{'Average': 56.88235294117647, '_id': {'Date': '26-3-2018'}}
{'Average': 51.0, '_id': {'Date': '29-3-2018'}}
{'Average': 48.72727272727273, '_id': {'Date': '31-3-2018'}}
{'Average': 45.2, '_id': {'Date': '2-4-2018'}}
{'Average': 58.44444444444444, '_id': {'Date': '3-4-2018'}}
{'Average': 62.57303370786517, '_id': {'Date': '4-4-2018'}}
{'Average': 53.142857142857146, '_id': {'Date': '5-4-2018'}}
{'Average': 61.71186440677966, '_id': {'Date': '6-4-2018'}}
{'Average': 50.69230769230769, '_id': {'Date': '7-4-2018'}}
{'Average': 60.75, '_id': {'Date': '8-4-2018'}}
{'Average': 52.69565217391305, '_id': {'Date': '12-4-2018'}}
{'Average': 58.57983193277311, '_id': {'Date': '13-4-2018'}

i.

In [36]:
cursor = climate_date.aggregate([
    {"$sort": {"GHI_w/m2": 1}},
    {"$limit": 10}
])

In [37]:
for document in cursor:
    pprint(document)

{'GHI_w/m2': 47,
 '_id': ObjectId('60aa1eb69343697468b4cd7c'),
 'air_temperature_celcius': 5,
 'date': '2-8-2018',
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 38.6,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('60aa1eb69343697468b4cd5b'),
 'air_temperature_celcius': 5,
 'date': '30-6-2018',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00A',
 'relative_humidity': 34.9,
 'windspeed_knots': 4.3}
{'GHI_w/m2': 49,
 '_id': ObjectId('60aa1eb69343697468b4cd5c'),
 'air_temperature_celcius': 5,
 'date': '1-7-2018',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 33.1,
 'windspeed_knots': 5.8}
{'GHI_w/m2': 56,
 '_id': ObjectId('60aa1eb69343697468b4cd66'),
 'air_temperature_celcius': 6,
 'date': '11-7-2018',
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.01G',
 'relative_humidity': 39.0,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 57,
 '_id': ObjectId('60aa1eb69343697468b4cd6a'),
 'air_temperature_celcius': 6,
 'date': '15-7-201

j.

In [38]:
cursor = climate_date.find(
    {"$or": [
        {"precipitation ":{"$regex": "^ 0.2.*G$"}},
        {"precipitation ":{"$regex": "^ 0.3[1|2|3|4|5]G$"}}
    ]}
)

In [39]:
for document in cursor:
    pprint(document)

{'GHI_w/m2': 157,
 '_id': ObjectId('60aa1eb69343697468b4ccb3'),
 'air_temperature_celcius': 19,
 'date': '13-1-2018',
 'max_wind_speed': 18.1,
 'precipitation ': ' 0.31G',
 'relative_humidity': 54.1,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('60aa1eb69343697468b4ccfe'),
 'air_temperature_celcius': 17,
 'date': '29-3-2018',
 'max_wind_speed': 21.0,
 'precipitation ': ' 0.24G',
 'relative_humidity': 49.9,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('60aa1eb69343697468b4cd14'),
 'air_temperature_celcius': 20,
 'date': '20-4-2018',
 'max_wind_speed': 15.9,
 'precipitation ': ' 0.31G',
 'relative_humidity': 53.5,
 'windspeed_knots': 7.2}
{'GHI_w/m2': 102,
 '_id': ObjectId('60aa1eb69343697468b4cd1a'),
 'air_temperature_celcius': 11,
 'date': '26-4-2018',
 'max_wind_speed': 20.0,
 'precipitation ': ' 0.24G',
 'relative_humidity': 40.8,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 122,
 '_id': ObjectId('60aa1eb69343697468b4cd39'),
 'air_temperature_celcius': 14,
 